In [1]:
import pandas as pd
import csv
from datetime import datetime
import time

In [2]:
train = pd.read_csv('train1.csv')
test = pd.read_csv('test.csv')
%time

CPU times: total: 0 ns
Wall time: 0 ns


In [63]:
# print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(1661795824262))))

OSError: [Errno 22] Invalid argument

In [13]:
# train.type.unique()

array(['0', 'carts', 'orders'], dtype=object)

In [16]:
# train['type'] = train['type'].str.replace('clicks','0', )
# train['type'] = train['type'].str.replace('carts','1', )
# train['type'] = train['type'].str.replace('orders','2', )

# test['type'] = test['type'].str.replace('clicks','0', )
# test['type'] = test['type'].str.replace('carts','1', )
# test['type'] = test['type'].str.replace('orders','2', )

In [17]:
# train.type.unique()

array(['0', '1', '2'], dtype=object)

In [65]:
train["minutes"] = train[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
train

,session,aid,ts,type,minutes
0,0,1517085,1659304800025,clicks,1.741433
1,0,1563459,1659304904511,clicks,1042.248583
2,0,1309446,1659367439426,clicks,4.676183
3,0,16246,1659367719997,clicks,2.522450
4,0,1781822,1659367871344,clicks,0.240867
...,...,...,...,...,...
39889595,999999,53921,1659464388426,clicks,0.567350
39889596,999999,729903,1659464422467,clicks,0.885833
39889597,999999,53921,1659464475617,clicks,5.888133
39889598,999999,1839535,1659464828905,clicks,24457.408350


In [20]:
train.head(10)

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks
5,0,1152674,1659367885796,clicks
6,0,1649869,1659369893840,carts
7,0,461689,1659369898050,carts
8,0,305831,1659370027105,orders
9,0,461689,1659370027105,orders


In [4]:
test

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks
...,...,...,...,...
928110,13099776,1159407,1661844072138,clicks
928111,13099776,546448,1661844142618,clicks
928112,13099777,468584,1661795832787,clicks
928113,13099778,926609,1661795832939,clicks


In [5]:
%%time
test_df = test
test_df["minutes"] = test[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
test_df = test_df.sort_values(['minutes'],ascending=False)    # 다음 행동까지 걸린 시간 내림차순 정렬


CPU times: total: 16 s
Wall time: 16 s


In [6]:
test_action_df = test_df.copy()
test_action_df.aid = ' ' + test_df.aid.astype('str')
test_action_df = test_action_df.groupby(['session','type'])['aid'].sum().reset_index()

# session type 으로 묶어서 aid 모아보기

In [7]:
next_orders_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
next_orders_df['type'] = 'orders'   ### carts인 것만 뽑아서 order하겠지? 하고 order로 바꿔버리기


next_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ])
next_carts_df['type'] = 'carts'     ### clicks인 것만 뽑아서 carts에 넣겠지? 하고 carts로 바꿔버리기


next_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ]).copy()
### click은 click인거지 하고 복사

In [8]:
next_orders_df = pd.merge(next_orders_df, next_clicks_df[['session', 'aid']], on ='session', how = 'left')
### order 에 click 인 aid 를 붙인다? 왜??
next_orders_df["aid"] = next_orders_df["aid_x"] + next_orders_df["aid_y"]
### aid 하나로 합친 다음
next_orders_df = next_orders_df.drop(['aid_x', 'aid_y'], axis =1)
### 합친 거만 남기고 기존 aid 는 삭제
next_orders_df

,session,type,aid
0,12899781,orders,199008 199008 194067 199008 199008 199008 199...
1,12899782,orders,1494780 834354 975116 127404 413962 595994 13...
2,12899786,orders,955252 955252
3,12899787,orders,1682750 1682750 1682750 1682750 1024433
4,12899790,orders,1830166 1219653 1830166
...,...,...,...
31227,13099762,orders,1154167 1154167 1154167 1154167 1154167 1154167
31228,13099764,orders,30503 1232089 373708 1424332 280211 456357 11...
31229,13099769,orders,1418917 405710 1418917 405710 1733323 405710 ...
31230,13099771,orders,1118750 1118750


In [9]:
recommend_df = pd.concat([next_orders_df, next_carts_df, next_clicks_df], axis =0)
### order cart click 아래로 다 합치기
recommend_df["session_type"] = recommend_df["session"].astype('str') + "_" + recommend_df["type"] 
### session_type 만들기 (제출하려고)
recommend_df

,session,type,aid,session_type
0,12899781,orders,199008 199008 194067 199008 199008 199008 199...,12899781_orders
1,12899782,orders,1494780 834354 975116 127404 413962 595994 13...,12899782_orders
2,12899786,orders,955252 955252,12899786_orders
3,12899787,orders,1682750 1682750 1682750 1682750 1024433,12899787_orders
4,12899790,orders,1830166 1219653 1830166,12899790_orders
...,...,...,...,...
236872,13099774,clicks,932098 1110447 575597 1611603 575597 1808266 ...,13099774_clicks
236874,13099775,clicks,3435,13099775_clicks
236875,13099776,clicks,675313 830008 255702 1159407 666859 554408 54...,13099776_clicks
236876,13099777,clicks,468584,13099777_clicks


In [41]:
best = train[train['type']=='orders']
order_count = {}
for i in best['aid']:
    order_count.setdefault(i,0)
    order_count[i] += 1
    
best = pd.DataFrame.from_dict(order_count, orient = 'index', columns=["count"])
best = best.sort_values('count',ascending=False)
best.head()
### order 가 가장 많은 아이템 순서 (index = aid)
### 80222 가 1위

,count
80222,943
166037,704
231487,646
1733943,583
1022566,577


In [50]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub.head()
## 제출 폼 샘플 데려오기

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [51]:
# sample_sub = pd.merge(sample_sub, recommend_df[["session_type","aid"]], on = "session_type", how ="left")
# sample_sub['next'] = sample_sub['aid'] + best_sold_list
# sample_sub['next'].fillna(best_sold_list, inplace = True)
# sample_sub['next'] = sample_sub['next'].str.strip()
# sample_sub = sample_sub.drop(["labels", "aid"], axis = 1)
# sample_sub.columns = ("session_type", "labels")
# sample_sub
### 이 방식은 결국 order 다음에 있는 cart 아이템, cart 다음에 있는 click 아이템을 추천하는 방식
### 추천이 최대 20개 까지 가능하므로 부족한 아이템은 베스트셀러로 채우는 방식

sample_sub = pd.merge(sample_sub, recommend_df[["session_type","aid"]], on = "session_type", how ="left")
# session_type 기준으로 뽑아둔 아이템 붙이기
sample_sub['aid'].fillna(80222, inplace = True)
# null 값은 가장 많이 팔린 80222로 대체
sample_sub = sample_sub.drop(['labels'], axis = 1)
# 기존 label 삭제
sample_sub.columns = ('session_type', 'labels')
# label 새로 붙이기
sample_sub.head()

,session_type,labels
0,12899779_clicks,59625
1,12899779_carts,59625
2,12899779_orders,80222
3,12899780_clicks,1142000 582732 973453 736515 1142000
4,12899780_carts,1142000 582732 973453 736515 1142000


In [52]:
sample_sub.to_csv('submission.csv', index=False)
# 제출할 파일 만들기